In [ ]:
from datasets import load_dataset
from IPython.display import display, Markdown
from collections import defaultdict
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# Dataset for ReflectionCoder
reflectioncoder_dataset = load_dataset("SenseLLM/ReflectionSeq-DS")

In [10]:
def _filter_python_only(row):
    for i, m in enumerate(row["messages"]):
        for j, part in enumerate(m["content"]):
            if "python" in part["content"]:
                return True
    return False


def _explode_individual_messages(batch):
    out = {
        "messages": [],
    }
    for i, sample in enumerate(batch["messages"]):
        all_parts = []
        for m in sample:
            for j, part in enumerate(m["content"]):
                all_parts.append(
                    {
                        "role": m["role"],
                        "content": part["content"],
                        "type": part["type"],
                    }
                )
        out["messages"].append(all_parts)
    return out


reflectioncoder_dataset = (
    reflectioncoder_dataset["train"]
    .filter(_filter_python_only)
    .map(_explode_individual_messages, batched=True)
    .remove_columns(["type"])
)
ds_df = reflectioncoder_dataset.to_polars()

In [ ]:
ds_df.describe()

In [ ]:
def _display_messages_markdown(df):
    messages = df["messages"]
    for k, row in enumerate(messages):
        display(Markdown(f"# Row {k}"))
        for i, m in enumerate(row):
            display(Markdown(f"#### Message {i}: {m['role']}, {m['type']}"))
            content = m["content"].replace("#", "\#").replace("\n", "  \n")
            display(Markdown(content))


def _get_all_part_types(df):
    type_cts = defaultdict(int)
    for row in df["messages"]:
        for i, m in enumerate(row):
            msg_type, msg_role = m["type"], m["role"]
            type_cts[f"{msg_role}_{msg_type}"] += 1
    return type_cts


_display_messages_markdown(ds_df[0:2])
# display(_get_all_part_types(ds_df[0:2]))

In [ ]:
from typing import List, Dict, Tuple


def process_reflection(sample: dict) -> Tuple[List[Dict[str, str]], Dict[str, str]]:
    # these fields are misnamed in the dataset
    for message in sample["messages"]:
        msg_role, msg_type = message["role"], message["type"]
        role_out = "assistant"
        print(message["role"], message["type"])

    return (
        conversation,
        {
            "role": "assistant",
            "content": markdown_json(sample_data),
        },
    )


display(process_reflection(reflectioncoder_dataset[0]))

In [8]:
reflection_dataset = load_dataset("glaiveai/reflection-v1")
reflection_dataset_pd = reflection_dataset["train"].to_polars()

In [ ]:
import polars as pl


def _display_reflection_v1_markdown(df: pl.DataFrame):
    for i, row in enumerate(df.to_dicts()):
        display(Markdown(f"# Row {i}"))
        display(Markdown(f"#### system: {row['system']}"))
        display(Markdown(f"#### Prompt: {row['prompt']}"))
        display(Markdown(f"#### Response: {row['response']}"))


_display_reflection_v1_markdown(reflection_dataset_pd[0:5])

In [8]:
humaneval_dataset = load_dataset("openai/openai_humaneval")

In [ ]:
import ast
from transformers.agents.python_interpreter import (
    evaluate_python_code,
    LIST_SAFE_MODULES,
)
import traceback

ALLOWED_FNS = {
    range,
    print,
    sum,
    enumerate,
    int,
    str,
    abs,
    zip,
    sorted,
    list,
    len,
    bin,
    isinstance,
    set,
    min,
    max,
    dict,
    filter,
    reversed,
    chr,
    ord,
    tuple,
    map,
    round,
}
ALLOWED_FN_DICT = {fn.__name__: fn for fn in ALLOWED_FNS}

# TODO execute tests
failed = []

SKIP_LIST = set(
    [
        "HumanEval/32",
        "HumanEval/38",
        "HumanEval/50",
        "HumanEval/75",
        "HumanEval/99",
        "HumanEval/104",
        "HumanEval/105",
        "HumanEval/126",
        "HumanEval/137",
        "HumanEval/145",
        "HumanEval/150",
        "HumanEval/160",
        "HumanEval/162",
    ]
)

for i, sample in enumerate(humaneval_dataset["test"]):
    prompt, solution, task_id = (
        sample["prompt"],
        sample["canonical_solution"],
        sample["task_id"],
    )
    tests, entrypoint = sample["test"], sample["entry_point"]
    if task_id in SKIP_LIST:
        continue
    prompt = prompt.replace(">>>", "\n")
    tests = tests.replace("candidate(", entrypoint + "(")
    full_code = prompt + solution + tests + "\ncheck()"
    # display(Markdown(f"```{full_code}```"))
    auth_imports = LIST_SAFE_MODULES + [
        "typing",
        "copy",
        "hashlib",
        "string",
        "collections",
    ]
    # print(auth_imports)
    try:
        fn = evaluate_python_code(
            full_code,
            ALLOWED_FN_DICT,
            authorized_imports=auth_imports,
        )
        print(f"result for {i}: {fn}")
    except Exception as e:
        # print(full_code)
        # traceback.print_exc()
        failed.append(sample["task_id"])

print(f"{len(failed)} / {len(humaneval_dataset['test'])} failed")
print(failed)

In [3]:
cc_dataset = load_dataset("deepmind/code_contests")["train"]

In [ ]:
test_counts = []
for row in tqdm(cc_dataset):
    n_public = len(row["public_tests"]["input"])
    n_private = len(row["private_tests"]["input"])
    test_counts.append((n_public, n_private))

In [ ]:
test_counts_np = np.array(test_counts)
print(test_counts_np.mean(axis=0), test_counts_np.std(axis=0))

plt.hist(test_counts_np[:, 0], bins=20)
plt.show()
plt.hist(test_counts_np[:, 1], bins=20)
plt.show()

In [32]:
codeforces_subset_pl = pl.read_parquet(
    "../dataset_samples/codeforces_problems_subset.parquet"
)

In [55]:
descriptions = codeforces_subset_pl[9:19]["description"].to_list()
for i, desc in enumerate(descriptions):
    display(Markdown(f"## Description {i}"))
    display(Markdown(desc))

## Description 0

Polycarp plays a computer game. In this game, the players summon armies of magical minions, which then fight each other.

Polycarp can summon n different minions. The initial power level of the i-th minion is a_i, and when it is summoned, all previously summoned minions' power levels are increased by b_i. The minions can be summoned in any order.

Unfortunately, Polycarp cannot have more than k minions under his control. To get rid of unwanted minions after summoning them, he may destroy them. Each minion can be summoned (and destroyed) only once.

Polycarp's goal is to summon the strongest possible army. Formally, he wants to maximize the sum of power levels of all minions under his control (those which are summoned and not destroyed).

Help Polycarp to make up a plan of actions to summon the strongest possible army!

Input

The first line contains one integer T (1 ≤ T ≤ 75) — the number of test cases.

Each test case begins with a line containing two integers n and k (1 ≤ k ≤ n ≤ 75) — the number of minions availible for summoning, and the maximum number of minions that can be controlled by Polycarp, respectively.

Then n lines follow, the i-th line contains 2 integers a_i and b_i (1 ≤ a_i ≤ 10^5, 0 ≤ b_i ≤ 10^5) — the parameters of the i-th minion.

Output

For each test case print the optimal sequence of actions as follows:

Firstly, print m — the number of actions which Polycarp has to perform (0 ≤ m ≤ 2n). Then print m integers o_1, o_2, ..., o_m, where o_i denotes the i-th action as follows: if the i-th action is to summon the minion x, then o_i = x, and if the i-th action is to destroy the minion x, then o_i = -x. Each minion can be summoned at most once and cannot be destroyed before being summoned (and, obviously, cannot be destroyed more than once). The number of minions in Polycarp's army should be not greater than k after every action.

If there are multiple optimal sequences, print any of them.

Example

Input


3
5 2
5 3
7 0
5 0
4 0
10 0
2 1
10 100
50 10
5 5
1 5
2 4
3 3
4 2
5 1


Output


4
2 1 -1 5
1
2
5
5 4 3 2 1

Note

Consider the example test.

In the first test case, Polycarp can summon the minion 2 with power level 7, then summon the minion 1, which will increase the power level of the previous minion by 3, then destroy the minion 1, and finally, summon the minion 5. After this, Polycarp will have two minions with power levels of 10.

In the second test case, Polycarp can control only one minion, so he should choose the strongest of them and summon it.

In the third test case, Polycarp is able to summon and control all five minions.

## Description 1

Easy and hard versions are actually different problems, so read statements of both problems completely and carefully.

Summer vacation has started so Alice and Bob want to play and joy, but... Their mom doesn't think so. She says that they have to read some amount of books before all entertainments. Alice and Bob will read each book together to end this exercise faster.

There are n books in the family library. The i-th book is described by three integers: t_i — the amount of time Alice and Bob need to spend to read it, a_i (equals 1 if Alice likes the i-th book and 0 if not), and b_i (equals 1 if Bob likes the i-th book and 0 if not).

So they need to choose some books from the given n books in such a way that:

  * Alice likes at least k books from the chosen set and Bob likes at least k books from the chosen set; 
  * the total reading time of these books is minimized (they are children and want to play and joy as soon a possible). 



The set they choose is the same for both Alice an Bob (it's shared between them) and they read all books together, so the total reading time is the sum of t_i over all books that are in the chosen set.

Your task is to help them and find any suitable set of books or determine that it is impossible to find such a set.

Input

The first line of the input contains two integers n and k (1 ≤ k ≤ n ≤ 2 ⋅ 10^5).

The next n lines contain descriptions of books, one description per line: the i-th line contains three integers t_i, a_i and b_i (1 ≤ t_i ≤ 10^4, 0 ≤ a_i, b_i ≤ 1), where:

  * t_i — the amount of time required for reading the i-th book; 
  * a_i equals 1 if Alice likes the i-th book and 0 otherwise; 
  * b_i equals 1 if Bob likes the i-th book and 0 otherwise. 

Output

If there is no solution, print only one integer -1. Otherwise print one integer T — the minimum total reading time of the suitable set of books.

Examples

Input

8 4
7 1 1
2 1 1
4 0 1
8 1 1
1 0 1
1 1 1
1 0 1
3 0 0


Output

18


Input

5 2
6 0 0
9 0 0
1 0 1
2 1 1
5 1 0


Output

8


Input

5 3
3 0 0
2 1 0
3 1 0
5 0 1
3 0 1


Output

-1

## Description 2

...

This is the easy version of the problem. The only difference is that in this version q = 1. You can make hacks only if both versions of the problem are solved.

There is a process that takes place on arrays a and b of length n and length n-1 respectively. 

The process is an infinite sequence of operations. Each operation is as follows: 

  * First, choose a random integer i (1 ≤ i ≤ n-1). 
  * Then, simultaneously set a_i = min\left(a_i, \frac{a_i+a_{i+1}-b_i}{2}\right) and a_{i+1} = max\left(a_{i+1}, \frac{a_i+a_{i+1}+b_i}{2}\right) without any rounding (so values may become non-integer). 

See notes for an example of an operation.

It can be proven that array a converges, i. e. for each i there exists a limit a_i converges to. Let function F(a, b) return the value a_1 converges to after a process on a and b.

You are given array b, but not array a. However, you are given a third array c. Array a is good if it contains only integers and satisfies 0 ≤ a_i ≤ c_i for 1 ≤ i ≤ n.

Your task is to count the number of good arrays a where F(a, b) ≥ x for q values of x. Since the number of arrays can be very large, print it modulo 10^9+7.

Input

The first line contains a single integer n (2 ≤ n ≤ 100).

The second line contains n integers c_1, c_2 …, c_n (0 ≤ c_i ≤ 100).

The third line contains n-1 integers b_1, b_2, …, b_{n-1} (0 ≤ b_i ≤ 100).

The fourth line contains a single integer q (q=1).

The fifth line contains q space separated integers x_1, x_2, …, x_q (-10^5 ≤ x_i ≤ 10^5).

Output

Output q integers, where the i-th integer is the answer to the i-th query, i. e. the number of good arrays a where F(a, b) ≥ x_i modulo 10^9+7.

Example

Input


3
2 3 4
2 1
1
-1


Output


56

Note

The following explanation assumes b = [2, 1] and c=[2, 3, 4] (as in the sample).

Examples of arrays a that are not good: 

  * a = [3, 2, 3] is not good because a_1 > c_1; 
  * a = [0, -1, 3] is not good because a_2 < 0. 



One possible good array a is [0, 2, 4]. We can show that no operation has any effect on this array, so F(a, b) = a_1 = 0.

Another possible good array a is [0, 1, 4]. In a single operation with i = 1, we set a_1 = min((0+1-2)/(2), 0) and a_2 = max((0+1+2)/(2), 1). So, after a single operation with i = 1, a becomes equal to [-1/2, 3/2, 4]. We can show that no operation has any effect on this array, so F(a, b) = -1/2.

## Description 8

Some country is populated by wizards. They want to organize a demonstration.

There are n people living in the city, x of them are the wizards who will surely go to the demonstration. Other city people (n - x people) do not support the wizards and aren't going to go to the demonstration. We know that the city administration will react only to the demonstration involving at least y percent of the city people. Having considered the matter, the wizards decided to create clone puppets which can substitute the city people on the demonstration. 

So all in all, the demonstration will involve only the wizards and their puppets. The city administration cannot tell the difference between a puppet and a person, so, as they calculate the percentage, the administration will consider the city to be consisting of only n people and not containing any clone puppets. 

Help the wizards and find the minimum number of clones to create to that the demonstration had no less than y percent of the city people.

Input

The first line contains three space-separated integers, n, x, y (1 ≤ n, x, y ≤ 104, x ≤ n) — the number of citizens in the city, the number of wizards and the percentage the administration needs, correspondingly.

Please note that y can exceed 100 percent, that is, the administration wants to see on a demonstration more people that actually live in the city ( > n).

Output

Print a single integer — the answer to the problem, the minimum number of clones to create, so that the demonstration involved no less than y percent of n (the real total city population). 

Examples

Input

10 1 14


Output

1


Input

20 10 50


Output

0


Input

1000 352 146


Output

1108

Note

In the first sample it is necessary that at least 14% of 10 people came to the demonstration. As the number of people should be integer, then at least two people should come. There is only one wizard living in the city and he is going to come. That isn't enough, so he needs to create one clone. 

In the second sample 10 people should come to the demonstration. The city has 10 wizards. They will all come to the demonstration, so nobody has to create any clones.

## Description 9

You are given an equation: 

Ax2 + Bx + C = 0. 

Your task is to find the number of distinct roots of the equation and print all of them in ascending order.

Input

The first line contains three integer numbers A, B and C ( - 105 ≤ A, B, C ≤ 105). Any coefficient may be equal to 0.

Output

In case of infinite root count print the only integer -1. In case of no roots print the only integer 0. In other cases print the number of root on the first line and the roots on the following lines in the ascending order. Print roots with at least 5 digits after the decimal point.

Examples

Input

1 -5 6


Output

2
2.0000000000
3.0000000000